# Connecting Python to SQL

In order to make this works correctly,
Please install first additional library to your python:
- mysql
- mysql connector

Details here if necessary:
https://pynative.com/python-mysql-database-connection/#h-arguments-required-to-connect

In [1]:
%pip install mysql
%pip install mysql-connector-python
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import mysql.connector
from mysql.connector import Error

### Creating here a specific function to get connected to a mysql server


In [3]:
# Definition of a function to get connected to the server
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name,
            ssl_ca = 'server-ca.pem',
            ssl_cert = 'client-cert.pem',
            ssl_key = 'client-key.pem'
        )
        print("MySQL Database connection successful")
    except Error as e:
        print("Error while connecting to MySQL", e)

    return connection

### Calling that function using host IP address, user name, password, etc
and uploading extracted data from the database into a pandas dataFrame

In [4]:
# Server information
host_name = '34.155.111.131'
user_name ='dmds'
user_password ='dmds_sql'
# As well as need to specify the name of the database to connect to
db_name ='Divvy'

# Calling the defined function
connection = create_db_connection(host_name, user_name, user_password, db_name)


MySQL Database connection successful


In [5]:
# Defining here the sql query you want to run
sql_query= """select * from Divvy.Trips"""

In [6]:
# Read the sql query and get access to the data from the database
if connection.is_connected():
    df = pd.read_sql(sql_query, connection)

# Display the content of the extracted data
df

C:\Users\morga\AppData\Local\Temp\ipykernel_9556\524196652.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, connection)


,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,to_station_id,usertype,gender,birthyear,age
0,17901375,2018-03-17 16:45:13,2018-03-17 16:48:12,5019,179,16,16,Subscriber,Male,1990,None
1,18744113,2018-06-03 10:09:37,2018-06-03 10:22:48,63,791,232,340,Subscriber,Male,1993,None
2,18916861,2018-06-13 06:16:29,2018-06-13 06:27:01,5178,632,74,140,Subscriber,Female,1988,None
3,19573259,2018-07-17 18:03:50,2018-07-17 18:15:59,362,729,43,35,Subscriber,Male,1976,None
4,18835275,2018-06-07 17:03:00,2018-06-07 17:20:35,478,1055,338,338,Customer,Male,1988,None
...,...,...,...,...,...,...,...,...,...,...,...
360303,19079616,2018-06-22 06:23:44,2018-06-22 06:28:47,1262,303,91,283,Subscriber,Male,1960,None
360304,21628759,2018-12-08 12:18:21,2018-12-08 12:32:36,6240,855,376,217,Subscriber,Male,1953,None
360305,18146104,2018-04-22 12:33:33,2018-04-22 12:42:31,6014,538,99,47,Customer,,0,None
360306,20241669,2018-08-18 22:58:21,2018-08-19 00:40:53,3768,6152,24,341,Customer,,0,None


In [7]:
# 3-1 plot X:age vs. Y:avg_trip_time
# remove trip_id duplicate
df = df.drop_duplicates(subset="trip_id")

# keep birthyear and tripduration
df3 = df[["birthyear", "tripduration"]]
df3.head(50)


# check null: no null
#print(df3.isna().sum())


# birthyear
# remove == 0
mask_y_0 = df3["birthyear"]!=0
df3 = df3[mask_y_0]


# trip range: 61-7173290
# remove trip>60*24*365
mask_t_minute = df3["tripduration"]<=60*24*365
# apply mask
df3 = df3[mask_t_minute]


# calculate age
df3["age"] = 2018-df3["birthyear"]
# remove age > 95
mask_y_95 = df3["age"]<=95
df3 = df3[mask_y_95].sort_values("age", ascending=True)
# get average duration per age
df3_1 = df3[["age", "tripduration"]].groupby("age")[["tripduration"]].mean()
df3_1

,tripduration
age,
15,463.000000
16,1427.232323
17,1751.880597
18,1203.189552
19,1015.258769
...,...
79,975.358974
80,589.000000
84,895.800000


In [8]:
# 3-2 groups by every 10 years of age
df3_2 = df3[["age", "tripduration"]]


# apply floor
df3_2["age_group"] = np.floor(df3_2["age"]/10)*10
# calculate avg duration for each age group
df3_2 = df3_2.groupby("age_group")["tripduration"].count()
df3_2


# export to excel
#with pd.ExcelWriter("Part3: age vs. duration.xlsx") as writer:
#    df3_1.to_excel(writer, sheet_name="age vs. duration")
#    df3_2.to_excel(writer, sheet_name="age_group_by_10 vs. duration")

age_group
10.0      2422
20.0    112182
30.0    106384
40.0     43024
50.0     29349
60.0     10209
70.0       862
80.0        17
Name: tripduration, dtype: int64